In [1]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse
from collections import namedtuple
from PIL import Image
# from object_detection.utils import dataset_util, label_map_util

In [2]:
def xml_to_csv(path):
    xml_list = []
    className = os.listdir(path)[1:]
    for i in className:
        for xml_file in glob.glob(path +"/"+i +'/*.xml'):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            filename = root.find('FileName').text
            xmin = int(root.find('DefectBound').find('X1').text)
            ymin = int(root.find('DefectBound').find('Y1').text)
            xmax = int(root.find('DefectBound').find('X2').text)
            ymax = int(root.find('DefectBound').find('Y2').text)
            width = str(int(xmax) - int(xmin))
            height = str(int(ymax) - int(ymin))
            value = (filename,
                    width,
                    height,
                    i,
                    xmin,
                    ymin,
                    xmax,
                    ymax,
                    )
            xml_list.append(value)


    column_name = ['filename', 'width', 'height',
                'className', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [3]:
def create_tf_example(group, path):
    print(os.path.join(path, '{}/{}.bmp'.format(group.object.className.values[0],group.filename)))
    with tf.io.gfile.GFile(os.path.join(path, '{}/{}.bmp'.format(group.object.className.values[0],group.filename)), 'rb') as fid:
        print(fid)
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['className'].encode('utf8'))
        classes.append(class_text_to_int(row['className']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [4]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [6]:
import tensorflow as tf
path = "東捷科技_20220504_ADC_釋出資料集_origin"
examples = xml_to_csv(path)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)


東捷科技_20220504_ADC_釋出資料集_origin\Udev/A114B068AG01001_Dft.bmp


NameError: name 'class_text_to_int' is not defined